In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests
from json import load
import json

## Open JSON file - Download file from link in Readme or Resources

In [2]:
#importing Json file
json_file_ghg ="global-greenhouse-gas-emissions.json"

In [3]:
gghg_file_df = pd.read_json(json_file_ghg)
gghg_file_df.head()

,datasetid,recordid,fields
0,global-greenhouse-gas-emissions,f54a80719f59eb970175add568e6ce7f33bfdd66,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
1,global-greenhouse-gas-emissions,706d949e60cc90591ca4f628818cf8d9b4f0c577,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
2,global-greenhouse-gas-emissions,939cf5d648c2b37258f0545860f9159de04babe3,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
3,global-greenhouse-gas-emissions,795477055ac695d8e8ddcd7f6dba993c1f3253bb,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
4,global-greenhouse-gas-emissions,a06fcfc69e16bcb8bbc892bc094f2cf12f54a77c,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."


In [4]:
gghg_fields_df = gghg_file_df.loc[:, "fields"]
gghg_fields_df.head()

0    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
1    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
2    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
3    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
4    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
Name: fields, dtype: object

## Make a Usable DataFrame

In [5]:
norm_gghg_fields = pd.json_normalize(gghg_fields_df, max_level = 2)
norm_gghg_fields.head()

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
0,Annex_I,6D,Bulgaria,0.025106,1985,N2O,BGR,Central Europe,Other waste handling
1,Annex_I,6D,Bulgaria,0.025106,2011,N2O,BGR,Central Europe,Other waste handling
2,Annex_I,6D,Bulgaria,0.025106,1990,N2O,BGR,Central Europe,Other waste handling
3,Annex_I,6D,Bulgaria,0.025106,1992,N2O,BGR,Central Europe,Other waste handling
4,Annex_I,6D,Bulgaria,0.025106,1995,N2O,BGR,Central Europe,Other waste handling


### Selecting Major 50 countries Representating Global Emissions DataBase

In [6]:
counrty_list = ['United States', 'Mexico', 'Brazil', 'Columbia', 'Venezuela','Chile','Peru','India','Bangladesh','Sri Lanka',
                'Myanmar', 'Pakistan', 'Russia', 'Ukraine', 'Kazakhstan', 'Azerbaijan', 'Armenia', 'Afghanistan', 'Uzbekistan',
                'South Africa', 'Nigeria', 'Ghana', 'Morocco', 'Egypt', 'Kenya', 'China', 'Japan', 'Hong Kong', 'Singapore', 
                'Thailand', 'Malaysia', 'Korea', 'Viet Nam', 'United Kingdom', 'France', 'Germany', 'Norway', 'Italy', 
                'Spain', 'Portugal', 'Sweden', 'Turkey', 'Greece', 'Saudi Arabia', 'Bahrain', 'Syrian Arab Republic', 
                'Iraq', 'Qatar', 'United Arab Emirates', 'Kuwait']

In [9]:
gghg_clist = norm_gghg_fields.loc[norm_gghg_fields["name"].isin(counrty_list)]
gghg_clist = gghg_clist.dropna(how = 'any')
gghg_clist.reset_index (drop = True, inplace = True)
gghg_clist

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
0,Non-Annex_I,1A3b,Armenia,0.044949,1990,N2O,ARM,Russia +,Road transportation
1,Non-Annex_I,1A3b,Armenia,0.000933,1996,N2O,ARM,Russia +,Road transportation
2,Non-Annex_I,1A3b,Armenia,0.029318,1977,N2O,ARM,Russia +,Road transportation
3,Non-Annex_I,1A3b,Armenia,0.022507,1973,N2O,ARM,Russia +,Road transportation
4,Non-Annex_I,1A3b,Armenia,0.019033,1971,N2O,ARM,Russia +,Road transportation
...,...,...,...,...,...,...,...,...,...
253350,Annex_I,1A4,United Kingdom,5120.000000,2011,CO2_org_short-cycle_C,GBR,OECD_Europe,Residential and other sectors
253351,Annex_I,1A4,United Kingdom,5930.000000,2010,CO2_org_short-cycle_C,GBR,OECD_Europe,Residential and other sectors
253352,Annex_I,1A4,United Kingdom,1190.000000,1990,CO2_org_short-cycle_C,GBR,OECD_Europe,Residential and other sectors
253353,Non-Annex_I,4F,Peru,2720.000000,2005,CO2_org_short-cycle_C,PER,Rest South America,Agricultural waste burning


### Delete rows to simplify Quering and plotting - The DF would provide a list of emissions per country for specific sector and year

In [10]:
del gghg_clist['ipcc_annex']
del gghg_clist['ipcc']
del gghg_clist['world_region']
del gghg_clist['iso_a3']
gghg_clist

,name,value,year,compound,ipcc_description
0,Armenia,0.044949,1990,N2O,Road transportation
1,Armenia,0.000933,1996,N2O,Road transportation
2,Armenia,0.029318,1977,N2O,Road transportation
3,Armenia,0.022507,1973,N2O,Road transportation
4,Armenia,0.019033,1971,N2O,Road transportation
...,...,...,...,...,...
253350,United Kingdom,5120.000000,2011,CO2_org_short-cycle_C,Residential and other sectors
253351,United Kingdom,5930.000000,2010,CO2_org_short-cycle_C,Residential and other sectors
253352,United Kingdom,1190.000000,1990,CO2_org_short-cycle_C,Residential and other sectors
253353,Peru,2720.000000,2005,CO2_org_short-cycle_C,Agricultural waste burning


### List of Vairous GH Gases in Global Dataset 

In [11]:
glob_emis_gases = gghg_clist['compound'].value_counts()
glob_emis_gases_df = pd.DataFrame(glob_emis_gases)
glob_emis_gases_df

,compound
CO2_excl_short-cycle_org_C,83043
N2O,80320
CH4,75985
CO2_org_short-cycle_C,14007


### Rename Organize and sort DF 

In [12]:
Rename_gghg_clist = gghg_clist.rename(columns = {"name": "Country", "value":"Global_Emission_ktonnes", "compound":"GH-gases", "ipcc_description": "Industry_Sector" })
organized_gghg_df = Rename_gghg_clist [["year", "Country", "GH-gases", "Industry_Sector", "Global_Emission_ktonnes"]]
organized_gghg_df = organized_gghg_df.sort_values('year')
organized_gghg_df.reset_index(drop=True, inplace=True)
organized_gghg_df

,year,Country,GH-gases,Industry_Sector,Global_Emission_ktonnes
0,1970,Hong Kong,CO2_excl_short-cycle_org_C,Manufacturing Industries and Construction,2538.160800
1,1970,Sri Lanka,CH4,Civil Aviation,0.000401
2,1970,Kazakhstan,CO2_excl_short-cycle_org_C,Lime production,738.000000
3,1970,Morocco,CH4,Enteric fermentation,280.256500
4,1970,Mexico,CO2_org_short-cycle_C,Residential and other sectors,19900.000000
...,...,...,...,...,...
253350,2018,Italy,CO2_excl_short-cycle_org_C,Lime production,1425.000000
253351,2018,Peru,CO2_excl_short-cycle_org_C,Inland navigation,937.237993
253352,2018,United Arab Emirates,CO2_excl_short-cycle_org_C,Other direct soil emissions,10.944300
253353,2018,Myanmar,CO2_excl_short-cycle_org_C,Solvent and other product use: chemicals,0.024448


In [13]:
organized_gghg_df.describe()

,Global_Emission_ktonnes
count,2.533550e+05
mean,8.154872e+03
std,7.899003e+04
min,-2.000000e-03
25%,2.585709e-01
50%,8.515735e+00
75%,5.167879e+02
max,4.585523e+06


In [14]:
organized_gghg_df.count()

year                       253355
Country                    253355
GH-gases                   253355
Industry_Sector            253355
Global_Emission_ktonnes    253355
dtype: int64

## Connect to Local SQL DataBase

In [15]:
from config import password


In [16]:
protocol = 'postgresql'
username = 'postgres'
host = 'localhost'
port = 5432
database_name = 'Global_Emissions'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
# connection = engine.connect()

In [17]:
engine.table_names()

['Global_oil_data_iea',
 'Global_oil_demand',
 'Global_Agri_Greenhouse_Emissions',
 'Global_Agri_Percent_Emissions',
 'Global_Greenhouse_Emissions']

###  Current writing to SQL is set at "replace" - use "append" to add to SLQ database for additional countries

In [18]:
organized_gghg_df.to_sql(name='Global_Greenhouse_Emissions', con=engine, if_exists='replace', index=False)

In [19]:
pd.read_sql_query('select * from "Global_Greenhouse_Emissions"', con=engine).head()

,year,Country,GH-gases,Industry_Sector,Global_Emission_ktonnes
0,1970,Hong Kong,CO2_excl_short-cycle_org_C,Manufacturing Industries and Construction,2538.160800
1,1970,Sri Lanka,CH4,Civil Aviation,0.000401
2,1970,Kazakhstan,CO2_excl_short-cycle_org_C,Lime production,738.000000
3,1970,Morocco,CH4,Enteric fermentation,280.256500
4,1970,Mexico,CO2_org_short-cycle_C,Residential and other sectors,19900.000000
